# Reduce light frame using master files

## import modules

우선 필요한 모듈을 import합니다.

In [4]:
#%%
from glob import glob
from pathlib import Path
import os
import numpy as np
import astropy.units as u

import ysfitsutilpy as yfu

import _astro_utilities
import _Python_utilities

import warnings
warnings.filterwarnings('ignore')

## 환경 설정

In [5]:
#%%
verbose = False
tryagain = False
trynightsky = True
#######################################################
BASEDIR = Path("/mnt/Rdata/ASTRO_data")  

PROJECDIR = BASEDIR / "C1-Variable"
TODODIR = PROJECDIR / "-_-_-_2017-01_-_RiLA600_STX-16803_-_2bin"

DOINGDIRs = sorted(_Python_utilities.getFullnameListOfsubDirs(TODODIR))
if verbose == True :
    print ("DOINGDIRs: ", format(DOINGDIRs))
    print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))

try : 
    BDFDIR = [x for x in DOINGDIRs if "CAL-BDF" in str(x)]
    if verbose == True :
        print ("BDFDIR: ", format(BDFDIR))
    BDFDIR = Path(BDFDIR[0])    
except : 
    BDFDIR = TODODIR
    pass

DOINGDIRs = sorted([x for x in DOINGDIRs if "_LIGHT_" in str(x)])
if verbose == True :
    print ("DOINGDIRs: ", DOINGDIRs)
    print ("len(DOINGDIRs): ", len(DOINGDIRs))
#######################################################

DOINGDIRs:  ['/mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2017-01_-_RiLA600_STX-16803_-_2bin/-_CAL-BDF_-_2017-01_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2017-01_-_RiLA600_STX-16803_-_2bin/BL-CAM_LIGHT_-_2017-01-03_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2017-01_-_RiLA600_STX-16803_-_2bin/BL-CAM_LIGHT_-_2017-01-04_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2017-01_-_RiLA600_STX-16803_-_2bin/BL-CAM_LIGHT_-_2017-01-10_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2017-01_-_RiLA600_STX-16803_-_2bin/BL-CAM_LIGHT_-_2017-01-12_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2017-01_-_RiLA600_STX-16803_-_2bin/BL-CAM_LIGHT_-_2017-01-13_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2017-01_-_RiLA600_STX-16803_-_2bin/BL-CAM_LIGHT_-_2017-01-14_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2017-01_-_RiLA600_STX-16803_

## reduce light frame

``DOINGDIRs``에 list 형태로 들어있는 디렉토리 경로안에 모든 파일들을 ``yfu.make_summary`` 함수를 이용하여 dataframe으로 만들어 파일명 하나하나 ``reduce``를 수행합니다. 

In [6]:
for DOINGDIR in DOINGDIRs[:1] :
    DOINGDIR = Path(DOINGDIR)
    if verbose == True :
        print(f"Starting: {str(DOINGDIR.parts[-1])}")
    
    MASTERDIR = BDFDIR / _astro_utilities.master_dir
    sMASTERDIR = DOINGDIR / _astro_utilities.master_dir
    REDUCEDDIR = DOINGDIR / _astro_utilities.reduced_dir
    REDUC_nightsky = DOINGDIR / _astro_utilities.reduced_nightsky_dir

    if not sMASTERDIR.exists():
        os.makedirs(str(sMASTERDIR))
        if verbose == True :
            print("{} is created...".format(str(sMASTERDIR)))

    if not REDUCEDDIR.exists():
        os.makedirs(str(REDUCEDDIR))
        if verbose == True :
            print("{} is created...".format(str(REDUCEDDIR)))

    if not REDUC_nightsky.exists():
        os.makedirs("{}".format(str(REDUC_nightsky)))
        if verbose == True :
            print("{} is created...".format(str(REDUC_nightsky)))

    summary = yfu.make_summary(DOINGDIR/"*.fit*")
    if summary is not None :
        if verbose == True :
            #print(summary)
            print("len(summary):", len(summary))
            print("summary:", summary)
            #print(summary["file"][0])

        df_light = summary.loc[summary["IMAGETYP"] == "LIGHT"].copy()
        df_light = df_light.reset_index(drop=True)

        summary_master = yfu.make_summary(MASTERDIR/"*.fit*", 
                           verbose = False,
                           )
        if verbose == True :
            print("summary_master", summary_master)

        summary_master_dark = summary_master.loc[summary_master["IMAGETYP"] == "DARK"].copy()
        summary_master_dark.reset_index(inplace=True)
        if verbose == True :
            print("summary_master_dark", summary_master_dark)

        if 'EXPTIME' in summary_master_dark :
            check_exptimes = summary_master_dark['EXPTIME'].drop_duplicates()
            check_exptimes = check_exptimes.reset_index(drop=True)
            if verbose == True :
                print("check_exptimes", check_exptimes)

        for _, row in df_light.iterrows():

            fpath = Path(row["file"])
            ccd = yfu.load_ccd(fpath)
            filt = ccd.header["FILTER"]
            expt = ccd.header["EXPTIME"]

            idx = abs(summary_master_dark['EXPTIME'] - expt).idxmin()
            if verbose == True :
                print(idx)

            if (REDUCEDDIR / fpath.name).exists() and tryagain == False :
                if verbose == True :
                    print(f"reduction file already exists...\n{fpath.name}")
                pass
            else :
                try : 
                    if not (MASTERDIR / f"master_flat_{filt.upper()}_norm.fits").exists() :
                        if verbose == True :
                            print(f"{MASTERDIR}/master_flat_{filt.upper()}_norm.fits is not exists...")
                    else :
                        if (MASTERDIR / f"master_dark_{expt:.0f}sec.fits").exists() :
                            if verbose == True :
                                print(f"Reduce with master_dark_{expt:.0f}sec.fits ...")

                            red = yfu.ccdred(
                                ccd,
                                output=Path(f"{REDUCEDDIR/ fpath.name}"),
                                mdarkpath=str(MASTERDIR / "master_dark_{:.0f}sec.fits".format(expt)),
                                mflatpath=str(MASTERDIR / "master_flat_{}_norm.fits".format(filt.upper())),
                                # flat_norm_value=1,  # 1 = skip normalization, None = normalize by mean
                                overwrite=True,
                                )
                        else : 
                            if verbose == True :
                                print(f"Reduce with master_dark_{summary_master_dark['EXPTIME'][idx]:.0f}sec.fits is not exists...")
                            red = yfu.ccdred(
                                ccd,
                                output=Path(f"{REDUCEDDIR/ fpath.name}"),
                                mdarkpath=str(MASTERDIR / f"master_dark_{summary_master_dark['EXPTIME'][idx]:.0f}sec.fits"),
                                mflatpath=str(MASTERDIR / f"master_flat_{filt.upper()}_norm.fits"),
                                dark_scale = True,
                                exptime_dark = summary_master_dark['EXPTIME'][idx],
                                # flat_norm_value=1,  # 1 = skip normalization, None = normalize by mean
                                overwrite=True,
                                )
                        if verbose == True :
                            print (f"Reduce Reduce {fpath.name} +++...")

                except Exception as e: 
                    print("err:", e)
                    pass
                
    if trynightsky == True : 
        REDUCNSKYDIR = DOINGDIR / _astro_utilities.reduced_nightsky_dir
        if not REDUCNSKYDIR.exists():
            os.makedirs("{}".format(str(REDUCNSKYDIR)))
            if verbose == True :
                print("{} is created...".format(str(REDUCNSKYDIR)))
    
        summary = yfu.make_summary(REDUCEDDIR /"*.fit*")
        if summary is not None :
            if verbose == True :
                print("len(summary):", len(summary))
                print("summary:", summary)
                #print(summary["file"][0])   

            df_light = summary.loc[summary["IMAGETYP"] == "LIGHT"].copy()
            df_light = df_light.reset_index(drop=True)

            if 'FILTER' in df_light :
                check_filters = df_light['FILTER'].drop_duplicates()
                check_filters = check_filters.reset_index(drop=True)
                if verbose == True :
                    print("check_filters", check_filters)

                for filt in check_filters:
                #for filt in ["V"]:
                    df_light_filt = df_light.loc[df_light["FILTER"] == filt].copy()
                    
                    if df_light_filt.empty:
                        if verbose == True :
                            print(f"The dataframe(df_light_filt) {filt} is empty")
                        pass
                    else:
                        if verbose == True :
                            print("len(df_light_filt):", len(df_light_filt))
                            print("df_light_filt:", df_light_filt)
                    
                        if (sMASTERDIR / f"nightskyflat-{filt}.fits").exists() and tryagain == False :
                            pass
                        else :      
                            File_Num = 80
                            if len(df_light_filt["file"]) > File_Num :
                                combine_lst = df_light_filt["file"].tolist()[:File_Num]
                            else : 
                                combine_lst = df_light_filt["file"].tolist()
                            if (sMASTERDIR / f"nightskyflat-{filt}_norm.fits").exists():
                                pass
                            else :
                                try : 
                                    ccd = yfu.imcombine(
                                                        combine_lst, 
                                                        combine="med",
                                                        scale="avg", 
                                                        scale_to_0th=False, #norm
                                                        reject="sc", 
                                                        sigma=2.5,
                                                        verbose=True,
                                                        memlimit = 2.e+11,
                                                        )
                                except :
                                    ccd = yfu.imcombine(
                                                        combine_lst, 
                                                        combine="med",
                                                        scale="avg", 
                                                        scale_to_0th=False, #norm
                                                        reject="sc", 
                                                        # sigma=2.5,
                                                        verbose=True,
                                                        memlimit = 2.e+11,
                                                        )
                                ccd.write(sMASTERDIR / f"nightskyflat-{filt}_norm.fits", overwrite=True)
                                if verbose == True :
                                    print (f"Create Create nightskyflat-{filt}_norm.fits +++...")

            for _, row in df_light.iterrows():
                fpath = Path(row["file"])
                ccd = yfu.load_ccd(REDUCEDDIR / fpath.name)
                filt = row["FILTER"]
                if (REDUCNSKYDIR / fpath.name).exists() and tryagain == False:
                    if verbose == True :
                        print(f"Nightsky reduction file already exists...\n{fpath.name}")
                    pass
                else :
                    try:    
                        ccd = yfu.ccdred(
                                        ccd, 
                                        mflatpath = sMASTERDIR / f"nightskyflat-{filt}_norm.fits",
                                        output = REDUCNSKYDIR / fpath.name
                                    )
                    except : 
                        # _Python_utilities.write_log(err_log_file, "FileNotFoundError") 
                        pass

Starting: BL-CAM_LIGHT_-_2017-01-03_-_RiLA600_STX-16803_-_2bin
All 98 keywords (guessed from /mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2017-01_-_RiLA600_STX-16803_-_2bin/BL-CAM_LIGHT_-_2017-01-03_-_RiLA600_STX-16803_-_2bin/BL-CAM_LIGHT_V_2017-01-03-12-16-01_100sec_RiLA600_STX-16803_-20c_2bin.fit) will be loaded.
len(summary): 80
summary:                                                  file  filesize  SIMPLE  \
0   /mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2017-0...   8449920    True   
1   /mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2017-0...   8449920    True   
2   /mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2017-0...   8449920    True   
3   /mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2017-0...   8449920    True   
4   /mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2017-0...   8449920    True   
..                                                ...       ...     ...   
75  /mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2017-0...   8449920    True   
76  /mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2017-0...   844992